
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [13]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

import pandas as pd

data = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1')
data = data[data['Sentiment'] != 'Neutral']

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
X = data['OriginalTweet'].to_numpy()
Y = data["Sentiment"].isin(["Positive", "Extremely Positive"]).astype(int).to_numpy()

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
# X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=5/8, random_state=42)

l = len(X)
len_train = int(0.6 * l)
len_test = int(0.15 * l)

X_train = X[:len_train]
Y_train = Y[:len_train]
X_test = X[len_train:len_train + len_test]
Y_test = Y[len_train:len_train + len_test]
X_valid = X[len_train + len_test:]
Y_valid = Y[len_train + len_test:]

In [14]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Define the pipeline
model = Pipeline([
    ("vect", CountVectorizer(
        stop_words="english"
    )),
    ("clf", LogisticRegression(
        solver="liblinear",     # liblinear is good for small datasets
        max_iter=500           # limit iterations for autograder to ensure convergence
    ))
])

model.fit(X_train, Y_train)

,steps,"[('vect', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [15]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

from sklearn.metrics import precision_score, recall_score

pred = model.predict(X_test)

precision = precision_score(Y_test, pred, average=None, labels=[0, 1])
recall = recall_score(Y_test, pred, average=None, labels=[0, 1])

precision_0 = precision[0]
precision_1 = precision[1]
recall_0 = recall[0]
recall_1 = recall[1]

In [16]:

# Part 4

import numpy as np

def cost(model,threshold,X,Y):
     # probability of positive class
    probs = model.predict_proba(X)[:, 1]
    
    preds = (probs >= threshold).astype(int)
    
    # calculate cost for each sample
    costs = np.where(
        (Y == 1) & (preds == 0), 1,
        np.where(
            (Y == 0) & (preds == 1), 5, 0)  
    )
    return np.mean(costs)

In [ ]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
# Define a range of thresholds to try
thresholds = np.linspace(0, 1, 101) 

c = [cost(model, t, X_test, Y_test) for t in thresholds]

optimal_threshold = thresholds[np.argmin(c)]
cost_at_optimal_threshold = min(c)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41
 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55
 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69
 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83
 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97
 0.98 0.99 1.  ]


In [ ]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

def calc_hoeffdings_interval(c, n, b =5, delta=0.01):
    epsilon =  b* np.sqrt(np.log(2/delta) / (2*n))
    lower_bound = c - epsilon
    upper_bound = c + epsilon
    return (lower_bound, upper_bound)

cost_interval_valid = calc_hoeffdings_interval(cost_at_optimal_threshold_valid, len(X_valid))

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [ ]:
# Part 7

from scipy.optimize import brentq

# Get probability predictions and apply optimal threshold
probs_valid = model.predict_proba(X_valid)[:, 1]
y_pred_valid = (probs_valid >= optimal_threshold).astype(int)

# Compute cost variable C for each sample
C = np.where(
    (Y_valid == 1) & (y_pred_valid == 0), 1,  # False negative
    np.where((Y_valid == 0) & (y_pred_valid == 1), 5, 0)  # False positive or correct
)

# Compute empirical mean and variance
mean_C = np.mean(C)
variance_of_C = np.var(C, ddof=0)

# Define Bennett's inequality confidence interval computation
def bennett_epsilon(variance, b, n, delta=0.01):
    if variance == 0:
        return 0
    
    def h(u):
        # Bennett's function h(u) = (1 + u) * ln(1 + u) - u
        return (1 + u) * np.log(1 + u) - u
    
    def equation(eps):
        # Equation to solve for epsilon
        return np.exp(-(n * variance / b**2) * h(b * eps / variance)) - delta
    
    return brentq(equation, 1e-12, b)

# Compute interval using Bennett's inequality
if variance_of_C == 0:
    interval_of_C = (mean_C, mean_C)
else:
    epsilon_bennett = bennett_epsilon(variance_of_C, 5, len(C))
    interval_of_C = (mean_C - epsilon_bennett, mean_C + epsilon_bennett)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)